# Exercise 2.2

## Goal

The performance of machine learning systems directly depends on the quality of input features. In this exercise, you will investigate the impact of individual features on a system for named entity recognition: what does the inclusion of each individual feature do to the results? And what happens when they are combined?

We saw in the lecture of Week 5 that the possibility of creating high quality word embeddings from large volumes of unlabelled data has improved results for many NLP tasks. In the second part of this exercise, we will illustrate this by comparing systems that use \`traditional\' feature values to word embeddings. 


## Acknowledgement

This exercise made use of examples from the following exercise (in the HLT course):

https://github.com/cltl/ma-hlt-labs/

Lab3: machine learning


## Procedure

This notebook will provide the code for running the experiments outlined above. You will only need to make minor adaptations to run the feature ablation analysis. As specified in Assignment 2, you will need to write a short report on your findings. The report should include tables presenting results accompanied by an explanation. You may limit the report to a description of the features (and why they may matter), the results and a brief conclusion (i.e. introduction, related work, methodological setup, explanation of the machine learning models chosen are not needed).

The notebooks and set up have been designed for educational purposes: design choices are based on clearly illustrating what is going on and facilitating the exercises.

## The Data

We will be working with the data from the CoNLL Named Entity Recognition shared task 2003. The data has been preprocessed to make some useful features directly availabe. It can be found in the Assignment 2.2.2 folder under data/.

We will be using the file conll_train_prep.txt for training and conll_test_prep.txt for testing.

The original data had the following (space separated) format:

Token POS-tag Chunklabel Goldlabel

The first lines of the conll file look like this:

-DOCSTART- -X- -X- O
  
EU NNP B-NP B-ORG

rejects VBZ B-VP O


A note on Chunklabel:
Chunks are phrases (or constituents), but unlike a full constituent parses that aim to provide the full sentence structure in constituents, chunkers provide shallow representations of a depth of maximal two. They typically return noun phrases, prepositional phrases and some times verb phrases (with no or few arguments). The labels indicate the type of phrase.

A note on the Goldlabel:
This is the correct interpretation according to human annotators. CoNLL follows the BIO labeling style for named entities as also illustrated in the Eisenstein chapter you read in Module 1.

During preprocessing, we added two columns and used tsv to represent the conll file. The format is:

Token  Preceding_token  Capitalization  POS-tag  Chunklabel  Goldlabel

The first lines look like this:

-DOCSTART-      FULLCAP -X-     -X-     O
  
EU              FULLCAP NNP     B-NP    B-ORG

rejects EU      LOWCASE VBZ     B-VP    O

Preceding_token: 
This column provides the token preceding the current token. (This is an empty space if there is no previous token).

Capitalization: 
This column provides information on the capitalization of the token.
It can take the following values:

* LOWCASE: token consists of lowercase letters only
* FULLCAP: token consists of uppercase letters only
* FIRSTCAP: the first letter of the token is in uppercase, the rest in lowercase
* NUMERIC: the token represents a numeric value
* PUNCT: the token is a punctuation marker
* MIXED: the token is a mixture of the above (all other)


## Packages

We will make use of the following packages:

* scikit-learn : provides lots of useful implementations for machine learning (and has relatively good documentation!)
* csv: a light-weight package to deal with data represented in csv (or related formats such as tsv)
* gensim: a useful package for working with word embeddings
* numpy: a packages that (among others) provides useful datastructures and operations to work with vectors

Some notes on design decisions (feel free to ignore these if this is all new to you):

* We are using csv rather than (the more common) pandas for working with the conll files, because pandas standardly applies type conversion, which we do not want when dealing with text that contains numbers (fixing this will make the code look more complex).
* scikit-learn provides several machine learning algorithms, but this is not the focus of this exercise. We are using logistic regression, because it serves the purpose of our experiments and is relatively efficient.



In [1]:
#this cell imports all the modules we'll need. Make sure to run this once before running the other cells


#sklearn is scikit-learn
import sklearn
import csv
import gensim
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Part 1: Traditional Features

In this first part, we will explore the impact of various features on named entity recognition.
We will use so-called traditional features, where the feature values (strings) are presented by one-hot encoding 

## Step 1: A Basic Classifier

We will first walk through the process of creating and evaluating a simple classifier that only uses the token itself as a feature. In the next step, we will run evaluations on this basic system.

This is generally a good way to start experimenting: first walk through the entire experimental process with a very basic, easy to create system to see if everything works, there are no problems with the data etc. You can then build up from there towards a more sophististicated system.


In [3]:
#functions for feature extraction and training a classifier


## For documentation on how to create input representations of features in scikit-learn:
# https://scikit-learn.org/stable/modules/feature_extraction.html

#Setting some variables that we will use multiple times
trainfile = '/Users/hernando/Desktop/NLP/port/assignment2/NLP_tech_distributional_semantics 2/data/conll_train_cap.txt'
testfile = '/Users/hernando/Desktop/NLP/port/assignment2/NLP_tech_distributional_semantics 2/data/conll_test_cap.txt'

def extract_features_token_only_and_labels(conllfile):
    '''Function that extracts features and gold label from preprocessed conll (here: tokens only).
    
    :param conllfile: path to the (preprocessed) conll file
    :type conllfile: string
    
    
    :return features: a list of dictionaries, with key-value pair providing the value for the feature `token' for individual instances
    :return labels: a list of gold labels of individual instances
    '''
    
    features = []
    labels = []
    conllinput = open(conllfile, 'r')
    #delimiter indicates we are working with a tab separated value (default is comma)
    #quotechar has as default value '"', which is used to indicate the borders of a cell containing longer pieces of text
    #in this file, we have only one token as text, but this token can be '"', which then messes up the format. We set quotechar to a character that does not occur in our file
    csvreader = csv.reader(conllinput, delimiter='\t',quotechar='|')
    for row in csvreader:
        #I preprocessed the file so that all rows with instances should contain 6 values, the others are empty lines indicating the beginning of a sentence
        if len(row) == 6:
            #structuring feature value pairs as key-value pairs in a dictionary
            #the first column in the conll file represents tokens
            feature_value = {'Token': row[0]}
            features.append(feature_value)
            #The last column provides the gold label (= the correct answer). 
            labels.append(row[-1])
    
    return features, labels



def create_vectorizer_and_classifier(features, labels):
    '''
    Function that takes feature-value pairs and gold labels as input and trains a logistic regression classifier
    
    :param features: feature-value pairs
    :param labels: gold labels
    :type features: a list of dictionaries
    :type labels: a list of strings
    
    :return lr_classifier: a trained LogisticRegression classifier
    :return vec: a DictVectorizer to which the feature values are fitted. 
    '''
    
    vec = DictVectorizer()
    #fit creates a mapping between observed feature values and dimensions in a one-hot vector, transform represents the current values as a vector 
    tokens_vectorized = vec.fit_transform(features)
    lr_classifier = LogisticRegression(solver='saga')
    lr_classifier.fit(tokens_vectorized, labels)
    
    return lr_classifier, vec

#extract features and labels:
feature_values, labels = extract_features_token_only_and_labels(trainfile) 
#create vectorizer and trained classifier:
lr_classifier, vectorizer = create_vectorizer_and_classifier(feature_values, labels)



## Step 2: Evaluation

We will now run a basic evaluation of the system on a test file. 
Two important properties of the test file:

1. the test file and training file are independent sets (if they contain identical examples, this is coincidental)
2. the test file is preprocessed in the exact same way as the training file 

The first function runs our classifier on the test data.

The second function prints out a confusion matrix (comparing predictions and gold labels per class). 
You can find more information on confusion matrices here: https://www.geeksforgeeks.org/confusion-matrix-machine-learning/

The third function prints out the macro precision, recall and f-score of the system

In [5]:

def get_predicted_and_gold_labels_token_only(testfile, vectorizer, classifier):
    '''
    Function that extracts features and runs classifier on a test file returning predicted and gold labels
    
    :param testfile: path to the (preprocessed) test file
    :param vectorizer: vectorizer in which the mapping between feature values and dimensions is stored
    :param classifier: the trained classifier
    :type testfile: string
    :type vectorizer: DictVectorizer
    :type classifier: LogisticRegression()
    
    
    
    :return predictions: list of output labels provided by the classifier on the test file
    :return goldlabels: list of gold labels as included in the test file
    '''
    
    #we use the same function as above (guarantees features have the same name and form)
    sparse_feature_reps, goldlabels = extract_features_token_only_and_labels(testfile)
    #we need to use the same fitting as before, so now we only transform the current features according to this mapping (using only transform)
    test_features_vectorized = vectorizer.transform(sparse_feature_reps)
    predictions = classifier.predict(test_features_vectorized)
    
    return predictions, goldlabels

def print_confusion_matrix(predictions, goldlabels):
    '''
    Function that prints out a confusion matrix
    
    :param predictions: predicted labels
    :param goldlabels: gold standard labels
    :type predictions, goldlabels: list of strings
    '''
    
    
    
    #based on example from https://datatofish.com/confusion-matrix-python/ 
    data = {'Gold':    goldlabels, 'Predicted': predictions    }
    df = pd.DataFrame(data, columns=['Gold','Predicted'])

    confusion_matrix = pd.crosstab(df['Gold'], df['Predicted'], rownames=['Gold'], colnames=['Predicted'])
    print (confusion_matrix)


def print_precision_recall_fscore(predictions, goldlabels):
    '''
    Function that prints out precision, recall and f-score
    
    :param predictions: predicted output by classifier
    :param goldlabels: original gold labels
    :type predictions, goldlabels: list of strings
    '''
    
    precision = metrics.precision_score(y_true=goldlabels,
                        y_pred=predictions,
                        average='macro')

    recall = metrics.recall_score(y_true=goldlabels,
                     y_pred=predictions,
                     average='macro')


    fscore = metrics.f1_score(y_true=goldlabels,
                 y_pred=predictions,
                 average='macro')

    print('P:', precision, 'R:', recall, 'F1:', fscore)
    
#vectorizer and lr_classifier are the vectorizer and classifiers created in the previous cell.
#it is important that the same vectorizer is used for both training and testing: they should use the same mapping from values to dimensions
predictions, goldlabels = get_predicted_and_gold_labels_token_only(testfile, vectorizer, lr_classifier)
print_confusion_matrix(predictions, goldlabels)
print_precision_recall_fscore(predictions, goldlabels)

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1173      12     79      2      9       2      8      3    380
B-MISC        10     390     15      1      2       8      1      2    273
B-ORG        115      36    681      8     10       8     44      2    757
B-PER         12       9     15    539      2       0      1     38   1001
I-LOC         19       4      1      1     95       4     24      1    108
I-MISC         0      11      0      0      1     117      3      0     84
I-ORG         41      10     38      1     34       5    280      3    423
I-PER          9       3     13     53      2       0      2    100    974
O              7      28      9      0      5      13     43      5  38444
P: 0.7662832549510706 R: 0.48137979346502646 F1: 0.564331316735006


## Step 3: A More Elaborate System

Now that we have run a basic experiment, we are going to investigate alternatives. In this exercise, we only focus on features. We will continue to use the same logistic regression classifier throughout the exercise.

We want to investigate the impact of individual features. We will thus use a function that allows us to specify whether we include a specific feature or not. The features we have at our disposal are:

* the token itself (as used above)
* the preceding token
* the capitalization indication (see above for values that this takes)
* the pos-tag of the token
* the chunklabel of the chunk the token is part of

In [15]:
feature_to_index = {'Token': 0, 'Prevtoken': 1, 'Cap': 2, 'Pos': 3, 'Chunklabel': 4}

In [7]:
# the functions with multiple features and analysis

#defines the column in which each feature is located (note: you can also define headers and use csv.DictReader)
feature_to_index = {'Token': 0, 'Prevtoken': 1, 'Cap': 2, 'Pos': 3, 'Chunklabel': 4}


def extract_features_and_gold_labels(conllfile, selected_features):
    '''Function that extracts features and gold label from preprocessed conll (here: tokens only).
    
    :param conllfile: path to the (preprocessed) conll file
    :type conllfile: string
    
    
    :return features: a list of dictionaries, with key-value pair providing the value for the feature `token' for individual instances
    :return labels: a list of gold labels of individual instances
    '''
    
    features = []
    labels = []
    conllinput = open(conllfile, 'r')
    #delimiter indicates we are working with a tab separated value (default is comma)
    #quotechar has as default value '"', which is used to indicate the borders of a cell containing longer pieces of text
    #in this file, we have only one token as text, but this token can be '"', which then messes up the format. We set quotechar to a character that does not occur in our file
    csvreader = csv.reader(conllinput, delimiter='\t',quotechar='|')
    for row in csvreader:
        #I preprocessed the file so that all rows with instances should contain 6 values, the others are empty lines indicating the beginning of a sentence
        if len(row) == 6:
            #structuring feature value pairs as key-value pairs in a dictionary
            #the first column in the conll file represents tokens
            feature_value = {}
            for feature_name in selected_features:
                row_index = feature_to_index.get(feature_name)
                feature_value[feature_name] = row[row_index]
            features.append(feature_value)
            #The last column provides the gold label (= the correct answer). 
            labels.append(row[-1])
    return features, labels

def get_predicted_and_gold_labels(testfile, vectorizer, classifier, selected_features):
    '''
    Function that extracts features and runs classifier on a test file returning predicted and gold labels
    
    :param testfile: path to the (preprocessed) test file
    :param vectorizer: vectorizer in which the mapping between feature values and dimensions is stored
    :param classifier: the trained classifier
    :type testfile: string
    :type vectorizer: DictVectorizer
    :type classifier: LogisticRegression()
    
    
    
    :return predictions: list of output labels provided by the classifier on the test file
    :return goldlabels: list of gold labels as included in the test file
    '''
    
    #we use the same function as above (guarantees features have the same name and form)
    features, goldlabels = extract_features_and_gold_labels(testfile, selected_features)
    #we need to use the same fitting as before, so now we only transform the current features according to this mapping (using only transform)
    test_features_vectorized = vectorizer.transform(features)
    predictions = classifier.predict(test_features_vectorized)
    
    return predictions, goldlabels


In [12]:
import itertools
#define which from the available features will be used (names must match key names of dictionary feature_to_index)
all_features = ['Token','Prevtoken','Cap','Pos','Chunklabel']

sparse_feature_reps, labels = extract_features_and_gold_labels(trainfile, all_features)
#we can use the same function as before for creating the classifier and vectorizer
lr_classifier, vectorizer = create_vectorizer_and_classifier(sparse_feature_reps, labels)
#when applying our model to new data, we need to use the same features
predictions, goldlabels = get_predicted_and_gold_labels(testfile, vectorizer, lr_classifier, all_features)
print_confusion_matrix(predictions, goldlabels)
print_precision_recall_fscore(predictions, goldlabels)

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1348      18    134     70      1       1      7     46     43
B-MISC        28     458     54     47      0       1      8     24     82
B-ORG        184      43   1038    175      1       1     26     57    136
B-PER         68      20     45   1258      0       0      1    141     84
I-LOC          5       0      0      0    155       4     40     30     23
I-MISC         1       8      0      3      3     128     12     23     38
I-ORG         21       8      5     11     45       8    521    111    105
I-PER          4       1      0     13      1       0     15   1093     29
O             29      41     70    101      3      37     36    105  38132
P: 0.775359110733929 R: 0.7352997074459117 F1: 0.7489500540371716


## Step 4: Feature Ablation Analysis

If all worked well, the system that made use of all features worked better than the system with just the tokens.
We now want to know which of the features contributed to this improved: do we want to include all features?
Or just some?

We can investigate this using *feature ablation analysis*. This means that we systematically test what happens if we add or remove a specific feature. Ideally, we investigate all possible combinations.

The cell below illustrates how you can use the code above to investigate a system with three features. You can modify the selected features to try out different combinations. You can either do this manually and rerun the cell or write a function that creates list of all combinations you want to tests and runs them one after the other.

Include your results in the report of this week.

In [17]:
sample2

[('Token', 'Prevtoken'),
 ('Token', 'Cap'),
 ('Token', 'Pos'),
 ('Token', 'Chunklabel'),
 ('Prevtoken', 'Cap'),
 ('Prevtoken', 'Pos'),
 ('Prevtoken', 'Chunklabel'),
 ('Cap', 'Pos'),
 ('Cap', 'Chunklabel'),
 ('Pos', 'Chunklabel')]

In [14]:

sample2=list(itertools.combinations(all_features,2))
for k in sample2:
    selected_features = list(k)
    feature_values, labels = extract_features_and_gold_labels(trainfile, selected_features)
    #we can use the same function as before for creating the classifier and vectorizer
    lr_classifier, vectorizer = create_vectorizer_and_classifier(feature_values, labels)
    #when applying our model to new data, we need to use the same features
    predictions, goldlabels = get_predicted_and_gold_labels(testfile, vectorizer, lr_classifier, selected_features)
    print_confusion_matrix(predictions, goldlabels)
    print_precision_recall_fscore(predictions, goldlabels)

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1231      13     54     10      2       1      3      0    354
B-MISC        11     392     18      3      0       0      1      0    277
B-ORG        102      36    785     17      0       0     14      0    707
B-PER         10       8     12    753      0       0      0      5    829
I-LOC          7       0      1      0    147       5     24      3     70
I-MISC         0       8      0      1      1     123      7      1     75
I-ORG         13       9      2      1     43       6    417      2    342
I-PER          3       0      5      7      1       0      5    462    673
O             12      29     30      4      3      16     20      3  38437
P: 0.8656238604796762 R: 0.5857941718382637 F1: 0.6851065088645175
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                             

In [18]:
sample3

[('Token', 'Prevtoken', 'Cap'),
 ('Token', 'Prevtoken', 'Pos'),
 ('Token', 'Prevtoken', 'Chunklabel'),
 ('Token', 'Cap', 'Pos'),
 ('Token', 'Cap', 'Chunklabel'),
 ('Token', 'Pos', 'Chunklabel'),
 ('Prevtoken', 'Cap', 'Pos'),
 ('Prevtoken', 'Cap', 'Chunklabel'),
 ('Prevtoken', 'Pos', 'Chunklabel'),
 ('Cap', 'Pos', 'Chunklabel')]

In [15]:

sample3=list(itertools.combinations(all_features,3))
for k in sample3:
    selected_features = list(k)
    feature_values, labels = extract_features_and_gold_labels(trainfile, selected_features)
    #we can use the same function as before for creating the classifier and vectorizer
    lr_classifier, vectorizer = create_vectorizer_and_classifier(feature_values, labels)
    #when applying our model to new data, we need to use the same features
    predictions, goldlabels = get_predicted_and_gold_labels(testfile, vectorizer, lr_classifier, selected_features)
    print_confusion_matrix(predictions, goldlabels)
    print_precision_recall_fscore(predictions, goldlabels)
    
    

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1349      14     76     51      2       1      9     54    112
B-MISC        23     436     51     37      0       0      3     35    117
B-ORG        167      46    983     71      1       1     20     76    296
B-PER         70      12     23   1152      0       0      1    151    208
I-LOC          9       0      1      0    157       4     34     29     23
I-MISC         1       9      0      2      3     128      9     25     39
I-ORG         27       9      5     10     45       9    518    107    105
I-PER          3       0      3     13      1       0     12   1093     31
O             33      32     43     53      2      37     31    118  38205
P: 0.7927099653832518 R: 0.7215971682331324 F1: 0.7475629935442316
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                             

P: 0.3262078243120412 R: 0.34754281555494837 F1: 0.27087557097750536


In [19]:
sample4

[('Token', 'Prevtoken', 'Cap', 'Pos'),
 ('Token', 'Prevtoken', 'Cap', 'Chunklabel'),
 ('Token', 'Prevtoken', 'Pos', 'Chunklabel'),
 ('Token', 'Cap', 'Pos', 'Chunklabel'),
 ('Prevtoken', 'Cap', 'Pos', 'Chunklabel')]

In [16]:
import itertools
sample4=list(itertools.combinations(all_features,4))
for k in sample4:
    selected_features = list(k)
    feature_values, labels = extract_features_and_gold_labels(trainfile, selected_features)
    #we can use the same function as before for creating the classifier and vectorizer
    lr_classifier, vectorizer = create_vectorizer_and_classifier(feature_values, labels)
    #when applying our model to new data, we need to use the same features
    predictions, goldlabels = get_predicted_and_gold_labels(testfile, vectorizer, lr_classifier, selected_features)
    print_confusion_matrix(predictions, goldlabels)
    print_precision_recall_fscore(predictions, goldlabels)

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1342      19    128     63      2       1      8     55     50
B-MISC        28     464     48     43      0       1      6     21     91
B-ORG        189      43   1045    154      1       1     23     68    137
B-PER         64      18     35   1259      0       0      2    147     92
I-LOC          9       0      1      0    155       4     34     28     26
I-MISC         1       9      0      2      3     128     10     22     41
I-ORG         26       8      5     11     44      10    517    109    105
I-PER          3       0      2     13      1       0     17   1089     31
O             33      37     70     88      3      38     32    111  38142
P: 0.7783202427869254 R: 0.7354987512056013 F1: 0.7501770883126218
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                             

In [20]:
sample1

[('Token',), ('Prevtoken',), ('Cap',), ('Pos',), ('Chunklabel',)]

In [13]:
sample1=list(itertools.combinations(all_features,1))
for k in sample1:
    selected_features = list(k)
    feature_values, labels = extract_features_and_gold_labels(trainfile, selected_features)
    #we can use the same function as before for creating the classifier and vectorizer
    lr_classifier, vectorizer = create_vectorizer_and_classifier(feature_values, labels)
    #when applying our model to new data, we need to use the same features
    predictions, goldlabels = get_predicted_and_gold_labels(testfile, vectorizer, lr_classifier, selected_features)
    print_confusion_matrix(predictions, goldlabels)
    print_precision_recall_fscore(predictions, goldlabels)

Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1173      12     79      2      9       2      8      3    380
B-MISC        10     390     15      1      2       8      1      2    273
B-ORG        115      36    681      8     10       8     44      2    757
B-PER         12       9     15    539      2       0      1     38   1001
I-LOC         19       4      1      1     95       4     24      1    108
I-MISC         0      11      0      0      1     117      3      0     84
I-ORG         41      10     38      1     34       5    280      3    423
I-PER          9       3     13     53      2       0      2    100    974
O              7      28      9      0      5      13     43      5  38444
P: 0.7662832549510706 R: 0.48137979346502646 F1: 0.564331316735006
Predicted  B-LOC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                     

# Part 2: One-hot versus Embeddings

In this second part of the exercise, we will compare results using one-hot encodings to pretrained word embeddings.

## One-hot representation

In one-hot representation, each feature value is represented by an n-dimensional vector, where n corresponds to the number of possible values the feature can take. In our system, the Token feature can take the value of each token that occurs at least once in the corpus. This feature thus uses a vector with the size of the vocabulary in the corpus. Each possible value is associated with a specific dimension. If this value is represented, that dimension will receive the value 1 and all other dimensions will have the value 0.

The system receive a concatenation of all feature representations as input.


## What does one-hot look like?

We will start with an illustration of a one-hot representation. We will use the capitalization feature for this: it has 6 possible values and is therefore represented by a 6-dimensional vector. If you would like a more precise look, you may consider creating a toy example of a few lines, in which the capitalization feature has different values.

In [21]:
# create classifier with caps feature only and print vectorizer, then with token only (but you see less)

selected_features = ['Cap']

feature_values, labels = extract_features_and_gold_labels(trainfile, selected_features)

#creating a vectorizing
vectorizer = DictVectorizer()
#fitting the values to dimensions (creating a mapping) and transforming the current observations according to this mapping
capitalization_vectorized = vectorizer.fit_transform(feature_values)
print(capitalization_vectorized.toarray())



[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


## Using word embeddings

We are now going to use word embeddings to represent tokens. We load a pretrained distributional semantic model.
You can use the same model as in Exercise 2.1. We tested the exercise with the same model (GoogleNews negative sampling 300 dimensions) as Exercise 2.1 as well.

Note: loading the model may take a while. You probably want to run that only once.


In [22]:
# this step takes a while
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/hernando/Desktop/NLP/Assignment2/NLP_tech_distributional_semantics/models/GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [23]:
def extract_embeddings_as_features_and_gold(conllfile,word_embedding_model):
    '''
    Function that extracts features and gold labels using word embeddings
    
    :param conllfile: path to conll file
    :param word_embedding_model: a pretrained word embedding model
    :type conllfile: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    
    :return features: list of vector representation of tokens
    :return labels: list of gold labels
    '''
    labels = []
    features = []
    
    conllinput = open(conllfile, 'r')
    csvreader = csv.reader(conllinput, delimiter='\t',quotechar='|')
    for row in csvreader:
        if len(row) == 6:
            if row[0] in word_embedding_model:
                vector = word_embedding_model[row[0]]
            else:
                vector = [0]*300
            features.append(vector)
            labels.append(row[-1])
    return features, labels

def create_classifier(features, labels):
    '''
    Function that creates classifier from features represented as vectors and gold labels
    
    :param features: list of vector representations of tokens
    :param labels: list of gold labels
    :type features: list of vectors
    :type labels: list of strings
    
    :returns trained logistic regression classifier
    '''
    
    
    lr_classifier = LogisticRegression(solver='saga')
    lr_classifier.fit(features, labels)
    
    return lr_classifier
    
    
def label_data_using_word_embeddings(testfile, word_embedding_model, classifier):
    '''
    Function that extracts word embeddings as features and gold labels from test data and runs a classifier
    
    :param testfile: path to test file
    :param word_embedding_model: distributional semantic model
    :param classifier: trained classifier
    :type testfile: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    :type classifier: LogisticRegression
    
    :return predictions: list of predicted labels
    :return labels: list of gold labels
    '''
    
    dense_feature_representations, labels = extract_embeddings_as_features_and_gold(testfile,word_embedding_model)
    predictions = classifier.predict(dense_feature_representations)
    
    return predictions, labels


# I printing announcements of where the code is at (since some of these steps take a while)

print('Extracting dense features...')
dense_feature_representations, labels = extract_embeddings_as_features_and_gold(trainfile,word_embedding_model)
print('Training classifier....')
classifier = create_classifier(dense_feature_representations, labels)
print('Running evaluation...')
predicted, gold = label_data_using_word_embeddings(testfile, word_embedding_model, classifier)
print_confusion_matrix(predictions, goldlabels)
print_precision_recall_fscore(predicted, gold)

Extracting dense features...
Training classifier....
Running evaluation...
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC        754      32    436    259      0       1      3     67    116
B-MISC        48     309    124     45      0       2      8     40    126
B-ORG        216      47    953    138      0       0      6     67    234
B-PER        135      20    273    938      0       1      1    154     95
I-LOC          4       7      0      0    131       7     42     38     28
I-MISC         1       2      0      5      3      94     22     38     51
I-ORG         26       9      4     24     51      16    443    148    114
I-PER          0       1      0     11      1       0     18   1084     41
O            157     108    283    129     11      44     50    110  37662
P: 0.6724982682447125 R: 0.6344319254956768 F1: 0.6517226942239397


## Including the preceding token

We can include the preceding token as a feature in a similar way. We simply concatenate the two vectors.

In [24]:

def extract_embeddings_of_current_and_preceding_as_features_and_gold(conllfile,word_embedding_model):
    '''
    Function that extracts features and gold labels using word embeddings for current and preceding token
    
    :param conllfile: path to conll file
    :param word_embedding_model: a pretrained word embedding model
    :type conllfile: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    
    :return features: list of vector representation of tokens
    :return labels: list of gold labels
    '''
    labels = []
    features = []
    
    conllinput = open(conllfile, 'r')
    csvreader = csv.reader(conllinput, delimiter='\t',quotechar='|')
    for row in csvreader:
        if len(row) == 6:
            if row[0] in word_embedding_model:
                vector1 = word_embedding_model[row[0]]
            else:
                vector1 = [0]*300
            if row[1] in word_embedding_model:
                vector2 = word_embedding_model[row[1]]
            else:
                vector2 = [0]*300
            features.append(np.concatenate((vector1,vector2)))
            labels.append(row[-1])
    return features, labels
    
    
def label_data_using_word_embeddings_current_and_preceding(testfile, word_embedding_model, classifier):
    '''
    Function that extracts word embeddings as features (of current and preceding token) and gold labels from test data and runs a trained classifier
    
    :param testfile: path to test file
    :param word_embedding_model: distributional semantic model
    :param classifier: trained classifier
    :type testfile: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    :type classifier: LogisticRegression
    
    :return predictions: list of predicted labels
    :return labels: list of gold labels
    '''
    
    features, labels = extract_embeddings_of_current_and_preceding_as_features_and_gold(testfile,word_embedding_model)
    predictions = classifier.predict(features)
    
    return predictions, labels



print('Extracting dense features...')
features, labels = extract_embeddings_of_current_and_preceding_as_features_and_gold(trainfile,word_embedding_model)
print('Training classifier...')
#we can use the same function as for just the tokens itself
classifier = create_classifier(features, labels)
print('Running evaluation...')
predicted, gold = label_data_using_word_embeddings_current_and_preceding(testfile, word_embedding_model, classifier)
print_confusion_matrix(predictions, goldlabels)
print_precision_recall_fscore(predicted, gold)

Extracting dense features...
Training classifier...
Running evaluation...
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC        754      32    436    259      0       1      3     67    116
B-MISC        48     309    124     45      0       2      8     40    126
B-ORG        216      47    953    138      0       0      6     67    234
B-PER        135      20    273    938      0       1      1    154     95
I-LOC          4       7      0      0    131       7     42     38     28
I-MISC         1       2      0      5      3      94     22     38     51
I-ORG         26       9      4     24     51      16    443    148    114
I-PER          0       1      0     11      1       0     18   1084     41
O            157     108    283    129     11      44     50    110  37662
P: 0.7706433909643632 R: 0.7554149229938387 F1: 0.7620927927958481


## A mixed system

The code below combines traditional features with word embeddings. Note that we only include features with a limited range of possible values. Combining one-hot token representations (using highly sparse dimensions) with dense representations is generally not a good idea.

In [25]:


def extract_word_embedding(token, word_embedding_model):
    '''
    Function that returns the word embedding for a given token out of a distributional semantic model and a 300-dimension vector of 0s otherwise
    
    :param token: the token
    :param word_embedding_model: the distributional semantic model
    :type token: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    
    :returns a vector representation of the token
    '''
    if token in word_embedding_model:
        vector = word_embedding_model[token]
    else:
        vector = [0]*300
    return vector


def extract_feature_values(row, selected_features):
    '''
    Function that extracts feature value pairs from row
    
    :param row: row from conll file
    :param selected_features: list of selected features
    :type row: string
    :type selected_features: list of strings
    
    :returns: dictionary of feature value pairs
    '''
    feature_values = {}
    for feature_name in selected_features:
        r_index = feature_to_index.get(feature_name)
        feature_values[feature_name] = row[r_index]
        
    return feature_values
    
    
def create_vectorizer_traditional_features(feature_values):
    '''
    Function that creates vectorizer for set of feature values
    
    :param feature_values: list of dictionaries containing feature-value pairs
    :type feature_values: list of dictionairies (key and values are strings)
    
    :returns: vectorizer with feature values fitted
    '''
    vectorizer = DictVectorizer()
    vectorizer.fit(feature_values)
    
    return vectorizer
        
    
def combine_sparse_and_dense_features(dense_vectors, sparse_features):
    '''
    Function that takes sparse and dense feature representations and appends their vector representation
    
    :param dense_vectors: list of dense vector representations
    :param sparse_features: list of sparse vector representations
    :type dense_vector: list of arrays
    :type sparse_features: list of lists
    
    :returns: list of arrays in which sparse and dense vectors are concatenated
    '''
    
    combined_vectors = []
    sparse_vectors = np.array(sparse_features.toarray())
    
    for index, vector in enumerate(sparse_vectors):
        combined_vector = np.concatenate((vector,dense_vectors[index]))
        combined_vectors.append(combined_vector)
    return combined_vectors
    

def extract_traditional_features_and_embeddings_plus_gold_labels(conllfile, word_embedding_model, vectorizer=None):
    '''
    Function that extracts traditional features as well as embeddings and gold labels using word embeddings for current and preceding token
    
    :param conllfile: path to conll file
    :param word_embedding_model: a pretrained word embedding model
    :type conllfile: string
    :type word_embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors
    
    :return features: list of vector representation of tokens
    :return labels: list of gold labels
    '''
    labels = []
    dense_vectors = []
    traditional_features = []
    
    conllinput = open(conllfile, 'r')
    csvreader = csv.reader(conllinput, delimiter='\t',quotechar='|')
    for row in csvreader:
        if len(row) == 6:
            token_vector = extract_word_embedding(row[0], word_embedding_model)
            pt_vector = extract_word_embedding(row[1], word_embedding_model)
            dense_vectors.append(np.concatenate((token_vector,pt_vector)))
            #mixing very sparse representations (for one-hot tokens) and dense representations is a bad idea
            #we thus only use other features with limited values
            other_features = extract_feature_values(row, ['Cap','Pos','Chunklabel'])
            traditional_features.append(other_features)
            #adding gold label to labels
            labels.append(row[-1])
            
    #create vector representation of traditional features
    if vectorizer is None:
        #creates vectorizer that provides mapping (only if not created earlier)
        vectorizer = create_vectorizer_traditional_features(traditional_features)
    sparse_features = vectorizer.transform(traditional_features)
    combined_vectors = combine_sparse_and_dense_features(dense_vectors, sparse_features)
    
    return combined_vectors, vectorizer, labels

def label_data_with_combined_features(testfile, classifier, vectorizer, word_embedding_model):
    '''
    Function that labels data with model using both sparse and dense features
    '''
    feature_vectors, vectorizer, goldlabels = extract_traditional_features_and_embeddings_plus_gold_labels(testfile, word_embedding_model, vectorizer)
    predictions = classifier.predict(feature_vectors)
    
    return predictions, goldlabels


print('Extracting Features...')
feature_vectors, vectorizer, gold_labels = extract_traditional_features_and_embeddings_plus_gold_labels(trainfile, word_embedding_model)
print('Training classifier....')
lr_classifier = create_classifier(feature_vectors, gold_labels)
print('Running the evaluation...')
predictions, goldlabels = label_data_with_combined_features(testfile, lr_classifier, vectorizer, word_embedding_model)
print_confusion_matrix(predictions, goldlabels)
print_precision_recall_fscore(predictions, goldlabels)

Extracting Features...
Training classifier....
Running the evaluation...
Predicted  B-LOC  B-MISC  B-ORG  B-PER  I-LOC  I-MISC  I-ORG  I-PER      O
Gold                                                                      
B-LOC       1423      35    150     22      3       2      7      4     22
B-MISC        26     515     57     25      0       5      4      0     70
B-ORG        234      75   1181     66      1       1     26      3     74
B-PER         21      14     55   1462      0       3      2     30     30
I-LOC          5       1      2      1    176       8     46      7     11
I-MISC         1       5      1      3      4     133     24      2     43
I-ORG         18      23     19      3     63      28    558     30     93
I-PER          0       9      2     32     10       1     31   1055     16
O             19      59     82     46     13      68    107     16  38144
P: 0.7718934049584222 R: 0.7858579015591616 F1: 0.7782709946991507
